## Introduction to `robotoc` 
# 5: Optimal Control of a Humanoid

## 本章の目的
ここでは，ヒューマノイドロボット iCub の最適制御問題（OCP）を定式化し，解くことが目的です．

まず， iCub の `robotoc.Robot` モデルを作成します．

In [ ]:
import robotoc

model_info = robotoc.RobotModelInfo()
model_info.urdf_path = 'urdf/icub_description/urdf/icub.urdf'
model_info.base_joint_type = robotoc.BaseJointType.FloatingBase
baumgarte_time_step = 0.05
model_info.surface_contacts = [robotoc.ContactModelInfo('l_sole', baumgarte_time_step),
                               robotoc.ContactModelInfo('r_sole', baumgarte_time_step)]
icub = robotoc.Robot(model_info)

## Cost Function（評価関数）
次に，評価関数 (cost function) を作成します．   
`robotoc`では，様々な評価関数のコンポーネント（**cost components**）を組み合わせて評価関数, `robotoc.CostFunction` (`std::shared_ptr<robotoc::CostFunction>` in C++), を作ります．  
各 **cost component** はある特定の評価関数の項を表しており，`robotoc.CostFunctionComponentBase` (`robotoc::CostFunctionComponentBase` in C++) を継承しています.   
**cost component** を評価関数 `robotoc.CostFunction` に集め， 評価関数 `robotoc.CostFunction` をソルバーに渡します．   
このようにすることで，複数の**cost components** を簡単に組み合わせて評価関数が設計できるようになっています.

基本的な **cost components** は `robotoc` のパッケージに既に多数実装されています.  
この例では， configuration つまり関節角，角速度，トルクなどについての **cost component** を考えます．   
以下がその **cost component** です．

### Jumping Pattern (ジャンプパターン)
評価関数を作る前に，まずはジャンプのパターンを設定します．　　 

In [ ]:
import numpy as np
jump_length = np.array([0.5, 0, 0])
flying_time = 0.25
ground_time = 0.7

### Configuration Space Cost

レファレンスと重みパラメータを設定していきます．   
以下のような命名規則があります．   

- `q_ref`: configuration のレファレンス (目標値）
- `q_weight`: configuration の重みパラメータ
- `v_weight`: 一般化速度 (関節角速度) の重みパラメータ
- `a_weight`: 一般化加速度 (関節角加速度) の重みパラメータ
- `u_weight`: 関節トルクの重みパラメータ
- `q_weight_terminal`, `v_weight_terminal`: ホライゾン終端での configuration と一般化速度の重みパラメータ
- `q_weight_impact`, `v_weight_impact`: 衝突の瞬間での configuration と一般化速度の重みパラメータ
- `dv_weight_impact`: 衝突の瞬間での 一般化速度変化（状態ジャンプ）の重みパラメータ

In [ ]:
q_standing = np.array([0, 0, 0.592, 0, 0, 1, 0,
                       0.20944, 0.08727, 0, -0.1745, -0.0279, -0.08726, # left leg
                       0.20944, 0.08727, 0, -0.1745, -0.0279, -0.08726, # right leg
                       0, 0, 0, # torso
                       0, 0.35, 0.5, 0.5, 0, 0, 0, # left arm 
                       0, 0.35, 0.5, 0.5, 0, 0, 0]) # right arm 
q_ref = q_standing.copy()
q_ref[0:3] += jump_length
q_weight = np.array([0, 1, 1, 100, 100, 100, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 1, 1,
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 
                     0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])
q_weight_terminal = q_weight
v_weight = np.full(icub.dimv(), 1.0e-03)
a_weight = np.full(icub.dimv(), 1.0e-05)
q_weight_impact = 1.0 * q_weight
v_weight_impact = 1.0 * v_weight
config_cost = robotoc.ConfigurationSpaceCost(icub)
config_cost.set_q_ref(q_standing)
config_cost.set_q_weight(q_weight)
config_cost.set_q_weight_terminal(q_weight)
config_cost.set_q_weight_impact(q_weight_impact)
config_cost.set_v_weight(v_weight)
config_cost.set_v_weight_terminal(v_weight)
config_cost.set_v_weight_impact(v_weight_impact)
config_cost.set_a_weight(a_weight)

以上の **cost components** を`robotoc.CostFunction`にまとめます．

In [ ]:
cost = robotoc.CostFunction()
cost.push_back(config_cost)

## Constraints（制約）
制約についても評価関数と同様に作成します．   
`robotoc.Constraints` (`std::shared_ptr<robotoc::Constraints>` in C++) は **constraint components**のコレクションになっています.
各 **constraints component** はある特定の制約を表しており， `robotoc.ConstraintComponentBase` (`robotoc::ConstraintComponentBase` in C++) を継承しています．     
これにより，様々な **constraints components** を簡単に組み合わせて制約を設計できます.

基本的な **constraint components** のは既に `robotoc` に多数実装されています.  
この例では，関節角度，角速度，トルクの制約を考えます．     

In [ ]:
joint_position_lower  = robotoc.JointPositionLowerLimit(icub)
joint_position_upper  = robotoc.JointPositionUpperLimit(icub)
joint_velocity_lower  = robotoc.JointVelocityLowerLimit(icub)
joint_velocity_upper  = robotoc.JointVelocityUpperLimit(icub)
joint_torques_lower   = robotoc.JointTorquesLowerLimit(icub)
joint_torques_upper   = robotoc.JointTorquesUpperLimit(icub)

さらに，摩擦錐 (friction cone) 制約をつくります．    

In [ ]:
friction_cone = robotoc.FrictionCone(icub)

制約を作り，上記のコンポーネントを加えます．

In [ ]:
constraints = robotoc.Constraints()
constraints.push_back(joint_position_lower)
constraints.push_back(joint_position_upper)
constraints.push_back(joint_velocity_lower)
constraints.push_back(joint_velocity_upper)
constraints.push_back(joint_torques_lower)
constraints.push_back(joint_torques_upper)
constraints.push_back(friction_cone)

## Contact Sequence（接触系列）
**Contact sequence** (`robotoc.ContactSequence`) は **contact status**  (`robotoc.ContactStatus`) の系列です．   
**Contact status** は接触についての情報，どの接触がアクティブなのか？ や接触の位置など，を記述したものです.  
これをOCPソルバーに渡すことで，OCPソルバーは自動的に問題を contact sequence に合わせて定式化してくれます．

以下のように contact sequence を作ります．

In [ ]:
contact_sequence = robotoc.ContactSequence(icub)

ここから，逐次的に contact sequence を作り上げていきます．   
まずは，初期状態での `robotoc.ContactStatus`をつくります．   
この`robotoc.ContactStatus`に接触状況（接触がアクティブか否か，接触の位置）を記述します．   
初期状態ではロボットは `q_standing`の姿勢をとって直立しています．　  
つまり，全ての脚で接触はアクティブです．      
従って，以下のように記述します．

In [ ]:
icub.forward_kinematics(q_standing)
x3d0_L = icub.frame_placement('l_sole')
x3d0_R = icub.frame_placement('r_sole')
contact_placements = {'l_sole': x3d0_L, 'r_sole': x3d0_R} 
mu = 0.6
friction_coefficients = {'l_sole': mu, 'r_sole': mu} 
contact_status_standing = icub.create_contact_status()
contact_status_standing.activate_contacts(['l_sole', 'r_sole'])
contact_status_standing.set_contact_placements(contact_placements)
contact_status_standing.set_friction_coefficients(friction_coefficients)

これを用いて contact sequence を初期化します．

In [ ]:
contact_sequence.init(contact_status_standing)

次に，両足の接触がアクティブでないとき，つまりロボットが飛び上がった時の contact status を作ります．

In [ ]:
contact_status_flying = icub.create_contact_status()

こちらを contact sequence に加えます．(push_backします．)    
この例では, **状態・制御入力の軌道と同時に接触のタイミングも最適化します (the switching time optimization (STO) と呼ばれます)** .      
STOを行うためには， contact sequence　に push_back するときに次のように指定します．

In [ ]:
contact_sequence.push_back(contact_status_flying, ground_time, sto=True)

このとき指定した時刻（上の例では`ground_time`）はスイッチ時刻の初期推定解として使われます．


次に，両足は再び地面につきます．   
そして初期のロボットの状態と比べ，両足の接触位置は `jump_length` だけ移動しています.   

In [ ]:
contact_placements['l_sole'].trans = contact_placements['l_sole'].trans + jump_length
contact_placements['r_sole'].trans = contact_placements['r_sole'].trans + jump_length 
contact_status_standing.set_contact_placements(contact_placements)
contact_sequence.push_back(contact_status_standing, ground_time+flying_time, sto=True)

次に，ロボットは再び飛び上がります．   
すなわち，両足の接触は再びアクティブでなくなります．

In [ ]:
contact_sequence.push_back(contact_status_flying, 2*ground_time+flying_time, sto=True)

最後に，両足の接触は再びアクティブになります．   
そして，両足の位置は先ほど着地した位置からさらに `jump_length` だけ移動します．

In [ ]:
contact_placements['l_sole'].trans = contact_placements['l_sole'].trans + jump_length 
contact_placements['r_sole'].trans = contact_placements['r_sole'].trans + jump_length 
contact_status_standing.set_contact_placements(contact_placements)
contact_sequence.push_back(contact_status_standing, 2*ground_time+2*flying_time, sto=True)

## STO (Switching Time Optimization) Cost and Constraints 
今，我々は **STO (スイッチ時刻の最適化)** を考えています．   
このとき，STOについての評価関数， STO cost (`robotoc.STOCostFunction`) と STO についての制約， STO constraints (`robotoc.STOConstraints`) を作り，ソルバーに渡す必要があります.     
(これらを渡さないと，ソルバーはSTOをせず，スイッチ時刻を固定して状態と入力だけの最適化を行います．）

まず STO cost `robotoc.STOCostFunction` (`std::shared_ptr<robotoc::STOCostFunction>` in C++) を作ります.   
ここでは特に component を加えず，空の STO cost を作ります.

In [ ]:
sto_cost = robotoc.STOCostFunction()

次に， STO constraints, i.e., `robotoc.STOConstraints` (`std::shared_ptr<robotoc::STOConstraints>` in C++) を作ります.   
ここでは， **minimum dwell-time**, すなわち各接触 phase がアクティブな時間の最小値を設定します．      
これを適切に設定することは重要です.   
今考えている問題では， 4つのスイッチと 5つの phase があります．

In [ ]:
sto_constraints = robotoc.STOConstraints(minimum_dwell_times=[0.6, 0.2, 0.6, 0.2, 0.6])

## Optimal Control Problem (OCP) and Solver

ここでは最適制御問題 (OCP) とそのソルバーを作ります．  
まずはホライゾン長さ `T` と離散化のグリッドの数 `N` を設定しましょう． 

In [ ]:
dt = 0.02
T = 2*flying_time + 3*ground_time
N = int(np.floor(T/dt))

最適制御問題 (OCP), `robotoc.OCP` (`robotoc::OCP` in C++) を用いて問題を記述します.   

In [ ]:
ocp = robotoc.OCP(robot=icub, cost=cost, constraints=constraints, 
                  sto_cost=sto_cost, sto_constraints=sto_constraints, 
                  contact_sequence=contact_sequence, T=T, N=N)

ではこのOCPのソルバー`robotoc.OCPSolver` (`robotoc::OCPSolver` in C++)　を作ります．   
このソルバーは以下の特徴があります: 
- Direct multiple-shooting 法　と primal-dual interior point 法 （主双対内点法）
- Inverse dynamics を用いた **unconstrained** OCP　 (i.e., no contacts or floating bases) の効率的なアルゴリズム.
- 状態のみの等式制約のための制約変換法
- Riccati recursion によるニュートンステップの計算
- スイッチ時刻最適化可能な Riccati recursion アルゴリズム

種々のソルバーのオプションを `robotoc.SolverOptions` により設定できます．   
ここでは次のようなオプションを設定します．   
- `kkt_tol_mesh`: Mesh-refinement を行うための **KKT error** の閾値
- `max_dt_mesh`: Mesh-refinement を行うための離散時間ステップの閾値
- `initial_sto_reg_iter`: 指定した回数だけ初めの反復で STO に関するヒューリスティックな正則化を行います.
- `max_iter`: 最大反復回数.  
- `nthreads`: 並列計算で用いるスレッドの数.

In [ ]:
solver_options = robotoc.SolverOptions()
solver_options.kkt_tol_mesh = 0.1
solver_options.max_dt_mesh = T/N 
solver_options.max_iter = 350
solver_options.initial_sto_reg_iter = 10 
solver_options.nthreads = 4
ocp_solver = robotoc.OCPSolver(ocp=ocp, solver_options=solver_options)

## Solve the OCP

初期状態 (`q`, `v`) と初期時刻 (`t`) が以下で与えられるOCP（最適制御問題）を考えます．

In [ ]:
t = 0.
q = q_standing.copy()
v = np.zeros(icub.dimv())

まずはソルバーを初期化します．   
以下は初期推定解を上の`q`, `v`に設定します．   
これは非常にシンプルですが高速な収束に割と重要です．   
(例えば状態方程式を用いるといったような，より良い初期化の方法もあると思います．しかし適当な初期化でも direct multiple-shooting の収束性能は良く実装は面倒なため実装していません．)

In [ ]:
ocp_solver.discretize(t)
ocp_solver.set_solution("q", q)
ocp_solver.set_solution("v", v)
ocp_solver.init_constraints()

また，不等式制約に関してソルバーを初期化します．    
(具体的には，STO制約も含めた主双対内点法のスラック変数とラグランジュ乗数を初期化しますが，特に内部について意識する必要はありません).

In [ ]:
ocp_solver.init_constraints()

ソルバーの現在の推定解がどの程度最適解に近いか，ということを判断するために，本ソルバーでは **KKT error** という指標を用います．   
これは， Karush–Kuhn–Tucker (KKT) 条件（1次の最適性の必要条件）のl2ノルムです．(最適解でこれは0になります)      
まず，初期化後の KKT error を見てみます． 

In [ ]:
ocp_solver.KKT_error(t, q, v)

次にOCPを解きます．

In [ ]:
ocp_solver.solve(t, q, v)

KKT error がとても小さくなっていることが分かります．    
(具体的には，`robotoc.SolverOption`で設定した閾値より小さくなっています).

In [ ]:
ocp_solver.KKT_error(t, q, v)

また，各反復での KKT error やステップサイズといった収束に関する情報も以下で見れます

In [ ]:
stats = ocp_solver.get_solver_statistics()
print(stats)

In [ ]:
%matplotlib inline
plot_kkt = robotoc.utils.PlotConvergence()
plot_kkt.figsize = 12, 9
kkt_data = [e.kkt_error for e in ocp_solver.get_solver_statistics().performance_index]
ts_data = ocp_solver.get_solver_statistics().ts
plot_kkt.plot(kkt_data=kkt_data, ts_data=ts_data)

また，以下で最適化された軌道を可視化できます．   
`'gepetto'` を `viewer_type` に選べば， 接触力および friction cone 制約も可視化できます．

In [ ]:
viewer = robotoc.utils.TrajectoryViewer(model_info=model_info, viewer_type='gepetto')
viewer.set_contact_info(mu=mu)
viewer.display(ocp_solver.get_time_discretization(), 
               ocp_solver.get_solution('q'), 
               ocp_solver.get_solution('f', 'WORLD'))